<a href="https://colab.research.google.com/github/wannasmile/colab_code_note/blob/main/Pytorch009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn.functional as F
from torch import nn, optim
from torch.autograd import Variable

training_data = [("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
                 ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])]

word_to_idx = {}
tag_to_idx = {}
for context, tag in training_data:
    for word in context:
        if word not in word_to_idx:
            word_to_idx[word] = len(word_to_idx)
    for label in tag:
        if label not in tag_to_idx:
            tag_to_idx[label] = len(tag_to_idx)
alphabet = 'abcdefghijklmnopqrstuvwxyz'
character_to_idx = {}
for i in range(len(alphabet)):
    character_to_idx[alphabet[i]] = i


class CharLSTM(nn.Module):
    def __init__(self, n_char, char_dim, char_hidden):
        super(CharLSTM, self).__init__()
        self.char_embedding = nn.Embedding(n_char, char_dim)
        self.char_lstm = nn.LSTM(char_dim, char_hidden, batch_first=True)

    def forward(self, x):
        x = self.char_embedding(x)
        _, h = self.char_lstm(x)
        return h[0]


class LSTMTagger(nn.Module):
    def __init__(self, n_word, n_char, char_dim, n_dim, char_hidden, n_hidden,
                 n_tag):
        super(LSTMTagger, self).__init__()
        self.word_embedding = nn.Embedding(n_word, n_dim)
        self.char_lstm = CharLSTM(n_char, char_dim, char_hidden)
        self.lstm = nn.LSTM(n_dim + char_hidden, n_hidden, batch_first=True)
        self.linear1 = nn.Linear(n_hidden, n_tag)

    def forward(self, x, word):
        char = torch.FloatTensor()
        for each in word:
            char_list = []
            for letter in each:
                char_list.append(character_to_idx[letter.lower()])
            char_list = torch.LongTensor(char_list)
            char_list = char_list.unsqueeze(0)
            if torch.cuda.is_available():
                tempchar = self.char_lstm(Variable(char_list).cuda())
            else:
                tempchar = self.char_lstm(Variable(char_list))
            tempchar = tempchar.squeeze(0)
            char = torch.cat((char, tempchar.cpu().data), 0)
        if torch.cuda.is_available():
            char = char.cuda()
        char = Variable(char)
        x = self.word_embedding(x)
        x = torch.cat((x, char), 1)
        x = x.unsqueeze(0)
        x, _ = self.lstm(x)
        x = x.squeeze(0)
        x = self.linear1(x)
        y = F.log_softmax(x)
        return y


model = LSTMTagger(len(word_to_idx), len(character_to_idx), 10, 100, 50, 128, len(tag_to_idx))
if torch.cuda.is_available():
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-2)


def make_sequence(x, dic):
    idx = [dic[i] for i in x]
    idx = Variable(torch.LongTensor(idx))
    return idx


for epoch in range(300):
    print('*' * 10)
    print('epoch {}'.format(epoch + 1))
    running_loss = 0
    for data in training_data:
        word, tag = data
        word_list = make_sequence(word, word_to_idx)
        tag = make_sequence(tag, tag_to_idx)
        if torch.cuda.is_available():
            word_list = word_list.cuda()
            tag = tag.cuda()
        # forward
        out = model(word_list, word)
        loss = criterion(out, tag)
        #running_loss += loss.data[0]
        running_loss += loss.item()
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print('Loss: {}'.format(running_loss / len(data)))
print()
input = make_sequence("Everybody ate the apple".split(), word_to_idx)
if torch.cuda.is_available():
    input = input.cuda()

out = model(input, "Everybody ate the apple".split())
print(out)

**********
epoch 1
Loss: 1.0968608260154724
**********
epoch 2
Loss: 1.0926524996757507
**********
epoch 3
Loss: 1.0884774923324585
**********
epoch 4
Loss: 1.0843341946601868
**********
epoch 5
Loss: 1.0802211165428162
**********
epoch 6
Loss: 1.0761367678642273
**********
epoch 7
Loss: 1.0720799565315247
**********
epoch 8
Loss: 1.0680488348007202
**********
epoch 9
Loss: 1.0640424489974976
**********
epoch 10
Loss: 1.0600594282150269
**********
epoch 11
Loss: 1.0560985207557678
**********
epoch 12
Loss: 1.0521583557128906
**********
epoch 13
Loss: 1.0482377409934998
**********
epoch 14
Loss: 1.0443356037139893
**********
epoch 15
Loss: 1.0404508709907532
**********
epoch 16
Loss: 1.0365821719169617
**********
epoch 17
Loss: 1.0327286124229431
**********
epoch 18


<ipython-input-1-75081d979bf4>:68: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  y = F.log_softmax(x)


Loss: 1.0288891792297363
**********
epoch 19
Loss: 1.0250626802444458
**********
epoch 20
Loss: 1.0212481915950775
**********
epoch 21
Loss: 1.0174448490142822
**********
epoch 22
Loss: 1.0136515498161316
**********
epoch 23
Loss: 1.0098674297332764
**********
epoch 24
Loss: 1.0060915350914001
**********
epoch 25
Loss: 1.0023231208324432
**********
epoch 26
Loss: 0.9985613226890564
**********
epoch 27
Loss: 0.9948052167892456
**********
epoch 28
Loss: 0.9910540878772736
**********
epoch 29
Loss: 0.9873071014881134
**********
epoch 30
Loss: 0.9835636019706726
**********
epoch 31
Loss: 0.9798227548599243
**********
epoch 32
Loss: 0.9760839939117432
**********
epoch 33
Loss: 0.9723464548587799
**********
epoch 34
Loss: 0.9686095714569092
**********
epoch 35
Loss: 0.9648728370666504
**********
epoch 36
Loss: 0.9611354768276215
**********
epoch 37
Loss: 0.9573970139026642
**********
epoch 38
Loss: 0.9536568522453308
**********
epoch 39
Loss: 0.9499145150184631
**********
epoch 40
Loss: 0.94